# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Домашнее задание 11

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.


**Баллы за задание:**

* Задача 1 &mdash; 20 баллов
* Задача 2 &mdash; 30 баллов
* Задача 3 &mdash; 60 баллов
* Задача 4 &mdash; 50 баллов
* Задача 5 &mdash; 50 баллов





-----

In [ ]:
import scipy.stats as sps
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

from statsmodels.stats.proportion import proportion_confint

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', font_scale=1.3, palette='Set2')

In [ ]:
# Bot check

# HW_ID: phds_hw8
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

---
### Задача 1
#### **Профиль физика**

Рассмотрим [данные](https://drive.google.com/file/d/1qGNN9pcSg6safgxqvWeimQEPtlyggOYc/view?usp=share_link) из первой задачи прошлого домашнего задания, нам понадобится только поперечный импульс `pT` мюона $\pi^+$ (`id = 211`). В предположении, что дисперсия значений не зависит от типа мюона, посчитайте с помощью формулы `MDE`, сколько мюонов каждого из двух исследуемых типов надо измерить, чтобы задетектировать разницу в средних импульсах равной 0.02? Можно ли снизить это число и из каких соображений? Сделайте соотвветсвующие вычисления

Выгрузим данные.

In [ ]:
df = pd.read_csv('pi.csv')
df.head()

Выделим нужные нам.

In [ ]:
pT_plus = <...>

Посмотрим на гистограмму.

In [ ]:
plt.figure(figsize=(10, 5))

plt.hist(pT_plus, alpha=0.5, bins=100, density=True, label='$\pi^+$')

plt.legend()
plt.show()

По формуле для `MDE` вычислите необходимый размер выборки.

In [ ]:
alpha = 0.05  # уровень значимости
beta = 0.8  # мощности
eps = 0.02  # размер эффекта

# сумма квантилей
q = <...>

In [ ]:
var_0 = pT_plus.var()
n_0 = <...>

n_0, var_0

In [ ]:
pT_plus.describe()

Обрежем выбросы, чтобы уменьшить дисперсию

**Внимание! Удалять выбросы можно только на основе данных до эксперимента. Например, не рассматривать в эксперименте пользователя, на котором произошел выброс. Удаление пользователей по выбросам по экспериментальным данных может привести к большим ошибкам.**

In [ ]:
pT_plus = pT_plus[pT_plus < ...]

In [ ]:
var_0 = <...>
n_0 = <...>

n_0, var_0

Посмотрим на зависимость размера выборки в исследовании от размера эффекта. Какой минимальный эффект можно зафиксировать с нашей выборкой?

In [ ]:
eps = np.linspace(0.01, 0.1, 100)
n = <...>

plt.plot(eps, n, lw=3)
plt.xlabel('Размер эффекта (MDE)')
plt.ylabel('Размер выборки');

In [ ]:
<...> # минимальный эффект, который можно задетектировать с данной выборкой

**Вывод**

### Задача 1
#### **Профиль биология**

Рассмотрим данные [данные по медицинским страховкам](https://www.kaggle.com/datasets/gauravduttakiit/medical-insurance-cost?select=Train_Data.csv) из задачи 2 задания по AВ тестам. С помощью формулы для `MDE` оцените количество клиентов страховых компаний, на которых нужно провести AB-тест новых рекомендаций по употреблению витаминов, чтобы задетектировать снижение средней стоимости страховки на 600 долларов. Считайте, что компании не расширяют свою клиентскую базу и данные представляют всех клиентов, которые могут участвовать в тесте

In [ ]:
DATA_PATH = <...>

In [ ]:
data = pd.read_csv(DATA_PATH)['charges'].values
data

array([ 3279.86855   , 21454.49423864,  1720.3537    , ...,
       10602.385     ,  8976.14045154,  7027.69896823])

In [ ]:
data.shape

(3630,)

In [ ]:
alpha = 0.05  # уровень значимости
beta = 0.8  # мощности
eps = 600  # размер эффекта

# сумма квантилей
q = <...>

var_0 = data.var()
n_0 = <...>  # размер выборки

n_0, var_0

Достаточно ли у вас клиентов для исследования?

**Ответ:**


Посмотрим на распределение суммы страховки

In [ ]:
plt.hist(data, bins=100)
plt.xlabel('Сумма страховки')
plt.ylabel('Количество');

Видим, что есть выбросы, завышающие нам дисперсию.

In [ ]:
data_1 = <...>

In [ ]:
data_1.shape

**Внимание! Удалять выбросы можно только на основе данных до эксперимента. Например, не рассматривать в эксперименте клиента, на котором произошел выброс. Удаление клиентов по выбросам по экспериментальным данных может привести к большим ошибкам.**

Посчитаем, сколько нужно пользователей

In [ ]:
var_1 = data_1.var()
n_1 = <...>

n_1, var_1

С помощью удаления выбросов мы сократили требуемое количество клиентов пропорционально уменьшению дисперсии.

In [ ]:
n_0 / n_1, var_0 / var_1

Как зависит требуемый размер выборки в исследовании от размера эффекта? Постройте график размера выборки от `MDE` для ожидаемой в нашем исследовании дисперсии и сделайте выводы.

In [ ]:
eps = np.arange(200, 2000, 50)
n = <...>

plt.plot(eps, n, lw=3)
plt.xlabel('Размер эффекта (MDE)')
plt.ylabel('Размер выборки');

**Вывод:**

---
### Задача 2

Выведите формулу `MDE` для относительного t-теста.

---
### Задача 3
#### **Профиль физика**


Квантовые компьтеры состоят из кубитов, каждый из которых по-отдельности можно представить в виде единичного вектора на сфере. Считывание состояние кубита — это получение проекции этого вектора на ось $z$, причём считав состояние один раз мы получаем детерменированное значение: 0 или 1. Таким образом для получения значения проекции с определённой точностью мы усредняем некоторое количество считываний.

В данной задаче мы будем генерировать выборку из распределения Бернулли, симулируя проекции, получаемые от кубита. Наша задача — исследовать минимальный размер выборки для разных значений разделения состояний.


Выберите или оцените каким-нибудь способом:

1. Статистический критерий (название / односторонний или двусторонний / связные или независимые выборки);

2. Размеры A и B групп;

3. Желаемый минимальный детектируемый эффект;

4. Мощность критерия.

Обоснуйте свой выбор в каждом из пунктов.


> Статистический критерий (название / односторонний или двусторонний / связные или независимые выборки);

**Ответ:**


> Мощность критерия

**Ответ:**

> Желаемый минимальный детектируемый эффект;

> Размер А и В групп;

Будем подбирать по таблице `MDE`-мощность-размер выборки

In [ ]:
def get_mde(alpha, beta, sample_size, var_R, two_sided=False):
    '''Расчитывает MDE

    :param alpha: желаемая ошибка первого рода
    :param beta: желаемая мощность
    :param sample_size: размер выборок
    :param var_R: оценка дисперсии выборок
    :param two_sided: двусторонний ли критерий
    :returns: теоретический MDE
    '''

    a = <...>
    return <...>


def get_sample_size(alpha, beta, mde, var_R, two_sided=False):
    '''Расчитывает размер выборки для детектирование MDE

    :param alpha: желаемая ошибка первого рода
    :param beta: желаемая мощность
    :param mde: необходимый эффект
    :param var_R: дисперсии выборок
    :returns: необходимый размер выборки
    '''

    a = <...>
    return <...>

Создадим датасеты для разных значений проекций. Задайте размер выборок size=2000.

In [ ]:
data = np.array([<...> for i in np.linspace(0.3, 0.8, 6)]) # возьмём центральные значения проекций

Вычислим значения таблицы.

In [ ]:
mde_grid = np.arange(0.01, 0.1, 0.01).reshape(-1, 1)  # абсолютный MDE
beta_grid = np.arange(0.6, 1, 0.05)  # мощность
alpha = <...>
var = <...> # дисперсия

sample_sizes = <...>

Посмотрим для крайних и центрального значений, как выглядит таблица.

In [ ]:
pd.DataFrame(sample_sizes[0], columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))

In [ ]:
pd.DataFrame(sample_sizes[2], columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))

In [ ]:
pd.DataFrame(sample_sizes[5], columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))

Какой размер детектируемого эффекта мы возьмём и какой для него размер выборок?

**Ответ:**



Для выбранного критерия проведите АА-тест по сгенерированным данным.

In [ ]:
new_data = sps.bernoulli.rvs(0.5, size=<...>)

In [ ]:
sample_size = <...>
mde = <...>
alpha = <...>

In [ ]:
def absolute_ttest(x, y, alpha=0.05):
    '''
    Абсолютный t-test.

    Аргументы:
    x, y -- выборки одинакового размера
    alpha -- уровень значимости

    Возвращает:
    stat -- статистика критерия
    left_bound, right_bound -- границы дов. интервала
    pvalue
    '''

    n = len(x)
    x_mean = x.mean()
    y_mean = y.mean()

    stat = <...>
    var = <...>
    std = np.sqrt(var)

    z_stat = np.sqrt(n) * stat / std
    pvalue = 2*sps.norm.sf(np.abs(z_stat))

    q = sps.norm.ppf(1 - alpha/2)
    left_bound = <...>
    right_bound = <...>

    return stat, left_bound, right_bound, pvalue

In [ ]:
p_vals = []

for i in tqdm(range(2000)):

    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    p_val = <...>
    p_vals.append(p_val)

p_vals = np.array(p_vals)

In [ ]:
def draw_interval(
    real_alpha, left_alpha, right_alpha,
    alpha=0.05, figsize=(7, 2)
):
    '''
    Отрисовка интервала для реального уровня значимости
    real_alpha -- оценка уровня значимости
    left_alpha, right_alpha -- границы доверительного интервала
    alpha -- теоретический уровень значимости
    figsize -- размер фигуры matplotlib
    '''

    with sns.axes_style("whitegrid"):
        plt.figure(figsize=(6, 2))
        plt.hlines(0, 0, 1, color='black', lw=2, alpha=0.6)
        plt.vlines(alpha, -1, 1, color='red', lw=5, linestyle='--', alpha=0.6)
        plt.fill_between(
            [left_alpha, right_alpha], [0.15]*2, [-0.15]*2,
            color='green', alpha=0.6
        )
        plt.scatter(real_alpha, 0, s=300, marker='*', color='red')
        plt.xlim((min(alpha, left_alpha)-1e-3, max(alpha, right_alpha)+1e-3))
        plt.title(
            f'Доля отвержений = {100*real_alpha:.2f}%, '
            f'интервал ({100*left_alpha:.2f}%, {100*right_alpha:.2f}%)'
        )
        plt.ylim((-0.5, 0.5))
        plt.yticks([])
        plt.tight_layout()
        plt.show()

In [ ]:
real_alpha = (p_vals < alpha).mean()
print(f'Реальный уровень значимости: {real_alpha.round(2)}')
left_alpha, right_alpha = proportion_confint((p_vals < alpha).sum(), len(p_vals), method='wilson')
draw_interval(real_alpha, left_alpha, right_alpha, alpha=alpha)

Посмотрите на гистограмму pvalue.

Какие выводы о валидности выбранного критерия вы можете сделать?

**Ответ:**


In [ ]:
pvals = []

for _ in tqdm(range(2000)):
    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    sample2 = sample2 + <...>
    sample2 += sps.norm.rvs(size=len(sample2), scale=0.1)

    pvals.append(<...>)

pvals = np.array(pvals)

In [ ]:
print(f'Реальная мощность критерия: {(pvals < alpha).mean().round(3)}')

**Вывод**



### Задача 3
#### **Профиль биология**

В одной фармацевтической компании собираются проводить II фазу клинических испытаний нового лекарства, ослабляющего симптомы диабета. Перед командой аналитики стоит задача о корректном проведении AB-тестирования. У вас имеются исторические [данные по пациентам](https://www.kaggle.com/datasets/jillanisofttech/diabetes-disease-updated-dataset?select=diabetes.csv), страдающим диабетом. С помощью них проведите дизайн эксперимента.


Выберите или оцените каким-нибудь способом:

1. Таргет клинических испытаний (первичную контрольную точку): уровень глюкозы, инсулина, толщину кожи и тд.;

2. Статистический критерий (название / односторонний или двусторонний / связные или независимые выборки);

3. Размеры A и B групп;

4. Желаемый минимальный детектируемый эффект;

5. Мощность критерия.

Обоснуйте свой выбор в каждом из пунктов.

Все ли полученные показатели вас устраивают? Если нет, то какие могут быть способы их улучшить?

*Примечание.* Если кажется, что информации недостаточно, сделайте свои дополнительные предположения, коррелирующие с реальностью, и работайте с ними.


Для выбранного критерия проведите АА-тест по историческим данным. Если исторических данных оказалось недостаточно для вашей модели, придумайте способ решения проблемы.

In [ ]:
DATA_PATH =

In [ ]:
data = pd.read_csv(DATA_PATH)
data = data[data['Outcome']==1]
data

Выберите, обоснуйте и визуализируйте:

> Таргет клинических испытаний (первичную контрольную точку): уровень глюкозы, инсулина, толщину кожи и тд.;

**Ответ:**

In [ ]:
plt.figure(figsize=(10, 1.5))
sns.boxplot(<...>, orient='h')
plt.title(<...>)
plt.show()

> Статистический критерий (название / односторонний или двусторонний / связные или независимые выборки);

**Ответ:**

> Мощность критерия

**Ответ:**

> Желаемый минимальный детектируемый эффект;

> Размер А и В групп;

Будем подбирать по рассмотренной на семинаре таблице `MDE`-мощность-размер выборки

In [ ]:
def get_mde(alpha, beta, sample_size, var_R, two_sided=False):
    '''Расчитывает MDE

    :param alpha: желаемая ошибка первого рода
    :param beta: желаемая мощность
    :param sample_size: размер выборок
    :param var_R: оценка дисперсии выборок
    :param two_sided: двусторонний ли критерий
    :returns: теоретический MDE
    '''

    a = <...>
    return <...>


def get_sample_size(alpha, beta, mde, var_R, two_sided=False):
    '''Расчитывает размер выборки для детектирование MDE

    :param alpha: желаемая ошибка первого рода
    :param beta: желаемая мощность
    :param mde: необходимый эффект
    :param var_R: дисперсии выборок
    :returns: необходимый размер выборки
    '''

    a = <...>
    return <...>

In [ ]:
mde_grid = np.arange(0.01, 0.1, 0.01).reshape(-1, 1)  # относительный MDE
beta_grid = np.arange(0.6, 1, 0.05)  # мощность
alpha = <...>
var = <...>.var() * 2 / (<...>.mean()**2)  # относительная дисперсия

sample_sizes = get_sample_size(<...>)
pd.DataFrame(sample_sizes, columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))

**Ответ:**

Для выбранного критерия проведите АА-тест по историческим данным. Если исторических данных оказалось недостаточно для вашей модели, придумайте способ решения проблемы.

In [ ]:
sample_size = <...>
mde = <...>
alpha = <...>

In [ ]:
def relative_ttest(x, y, alpha=0.05):
    '''
    Относительный t-test.

    Аргументы:
    x, y -- выборки одинакового размера
    alpha -- уровень значимости

    Возвращает:
    stat -- статистика критерия
    left_bound, right_bound -- границы дов. интервала
    pvalue
    '''

    n = len(x)
    x_mean = x.mean()
    y_mean = y.mean()

    stat = <...>
    var = <...>
    std = np.sqrt(var)

    z_stat = np.sqrt(n) * stat / std
    pvalue = 2*sps.norm.sf(np.abs(z_stat))

    q = sps.norm.ppf(1 - alpha/2)
    left_bound = <...>
    right_bound = <...>

    return stat, left_bound, right_bound, pvalue

In [ ]:
p_vals = []

for i in tqdm(range(2000)):

    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    p_val = <...>
    p_vals.append(p_val)

p_vals = np.array(p_vals)

In [ ]:
def draw_interval(
    real_alpha, left_alpha, right_alpha,
    alpha=0.05, figsize=(7, 2)
):
    '''
    Отрисовка интервала для реального уровня значимости
    real_alpha -- оценка уровня значимости
    left_alpha, right_alpha -- границы доверительного интервала
    alpha -- теоретический уровень значимости
    figsize -- размер фигуры matplotlib
    '''

    with sns.axes_style("whitegrid"):
        plt.figure(figsize=(6, 2))
        plt.hlines(0, 0, 1, color='black', lw=2, alpha=0.6)
        plt.vlines(alpha, -1, 1, color='red', lw=5, linestyle='--', alpha=0.6)
        plt.fill_between(
            [left_alpha, right_alpha], [0.15]*2, [-0.15]*2,
            color='green', alpha=0.6
        )
        plt.scatter(real_alpha, 0, s=300, marker='*', color='red')
        plt.xlim((min(alpha, left_alpha)-1e-3, max(alpha, right_alpha)+1e-3))
        plt.title(
            f'Доля отвержений = {100*real_alpha:.2f}%, '
            f'интервал ({100*left_alpha:.2f}%, {100*right_alpha:.2f}%)'
        )
        plt.ylim((-0.5, 0.5))
        plt.yticks([])
        plt.tight_layout()
        plt.show()

In [ ]:
real_alpha = (p_vals < alpha).mean()
print(f'Реальный уровень значимости: {real_alpha.round(2)}')
left_alpha, right_alpha = proportion_confint((p_vals < alpha).sum(), len(p_vals), method='wilson')
draw_interval(real_alpha, left_alpha, right_alpha, alpha=alpha)

Посмотрите также на гистограмму p-value

Какие выводы о валидности выбранного критерия вы можете сделать?

**Ответ:**

Проверьте также мощность критерия на выбранном `MDE`.


In [ ]:
pvals = []

for _ in tqdm(range(2000)):
    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    sample2 = sample2 * <...>
    sample2 += sps.norm.rvs(size=len(sample2), scale=10)

    pvals.append(<...>)

pvals = np.array(pvals)

In [ ]:
print(f'Реальная мощность критерия: {(pvals < alpha).mean().round(3)}')

**Вывод**

---

### Задача 4
#### **Профиль физика**


Рассмотрим [данные о звездах](https://www.kaggle.com/datasets/fernandolima23/classification-in-asteroseismology?select=classification_in_asteroseismology.csv) из предыдущего домашнего задания:

Датасет содержит информацию о звездах, где `POP` - разделяет звёзды по классам. Нас будeт интересовать столбец `Dnu`.

Считайте данные.

In [ ]:
df = pd.read_csv("classification_in_asteroseismology.csv")

In [ ]:
df

Получите таблицу `MDE`-мощности-размера выборки для параметра `Dnu` как в 3-м задании, но теперь проведя относительный ttest и написав его функцию.

In [ ]:
def relative_ttest(x, y, alpha=0.05):
    '''
    Относительный t-test.

    Аргументы:
    x, y -- выборки одинакового размера
    alpha -- уровень значимости

    Возвращает:
    stat -- статистика критерия
    left_bound, right_bound -- границы дов. интервала
    pvalue
    '''

    n = len(x)
    x_mean = x.mean()
    y_mean = y.mean()

    stat = <...>
    var = <...>
    std = np.sqrt(var)

    z_stat = np.sqrt(n) * stat / std
    pvalue = 2*sps.norm.sf(np.abs(z_stat))

    q = sps.norm.ppf(1 - alpha/2)
    left_bound = <...>
    right_bound = <...>

    return stat, left_bound, right_bound, pvalue

In [ ]:
mde_grid = np.arange(0.01, 0.1, 0.01).reshape(-1, 1) # относительный MDE
beta_grid = np.arange(0.6, 1, 0.05)  # мощность
alpha = <...>
var = <...>.var() * 2 / (<...>.mean()**2)  # относительная дисперсия

sample_sizes = get_sample_size(<...>)
pd.DataFrame(sample_sizes, columns=beta_grid.reshape(-1), index=mde_grid.reshape(-1))

Какие значения `MDE`, достоверности и размера выборки стоит выбрать?

**Ответ:**


Далее как в 3-й задаче по физике посмотрим на реальный уровень значимости, гистограмму p_value и реальный уровень значимости.

In [ ]:
sample_size = <...>
mde = <...>
alpha = <...>

In [ ]:
p_vals = []

for i in tqdm(range(2000)):

    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    p_val = <...>
    p_vals.append(p_val)

p_vals = np.array(p_vals)

In [ ]:
real_alpha = (p_vals < alpha).mean()
print(f'Реальный уровень значимости: {real_alpha.round(2)}')
left_alpha, right_alpha = proportion_confint((p_vals < alpha).sum(), len(p_vals), method='wilson')
draw_interval(real_alpha, left_alpha, right_alpha, alpha=alpha)

Посмотрите также на гистограмму p-value.

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(10, 4))
plt.title('Гистограмма pvalue')
sns.histplot(p_vals)
plt.show()

Что можно сказать о валидности критерия?

**Ответ:**

Проверьте также мощность критерия на выбранном `MDE`.


In [ ]:
pvals = []

for _ in tqdm(range(2000)):
    sample1, sample2 = np.random.choice(<...>, (2, sample_size), replace=False)

    sample2 = sample2 * <...>
    sample2 += sps.norm.rvs(size=len(sample2), scale=0.1)

    pvals.append(<...>)

pvals = np.array(pvals)

In [ ]:
print(f'Реальная мощность критерия: {(pvals < alpha).mean().round(3)}')

Теперь воспользуемся стратификацией. Будем разбивать на страты по параметру `POP` выборки.

Реализуйте функции для стратифицированного семплирования и стратифицированного относительного t-test. Заранее вспомните с семинара, какое семплирование какому тесту подходит. Не забудьте о возможности пост-стратификации. Какие методы подойдут вам, учитывая количество страт и размер выборки?

**Ответ:**

In [ ]:
def calculate_strata_weights(data):
    '''
    Вычисляет веса страт.

    :param data: исходные данные
    :return: веса страт
    '''

    strata_weights = data.groupby(by='strata')['sample'].count()
    strata_weights = strata_weights / strata_weights.sum()
    return <...>


def weighted_mean(data, weights):
    '''
    Вычисляет взвешенное среднее.

    :param data: данные
    :param weights: веса страт
    :return: взвешенное среднее
    '''

    strata_means = data.groupby(by='strata')['sample'].mean()
    return <...>


def simple_mean(data, weights):
    '''
    Вычисляет обычное выборочное среднее

    :param data: данные
    :param weights: не используется, нужен для совместимости
    :return: выборочное среднее
    '''

    return <...>


def random_sampler(data, sample_size):
    '''
    Случайное семплирование двух групп одного размера из данных.

    :param data: исходные данные
    :param sample_size: размер группы
    :return: две группы данных
    '''

    choiced = data.sample(n=2 * sample_size)
    return choiced.iloc[:sample_size], choiced.iloc[sample_size:]

def stratified_sampler(data, sample_size):
    '''
    Стратифицированное семплирование выборок.

    :param data: данные
    :param sample_size: размер выборки
    :return: две выборки
    '''

    groupped = data.groupby(<...>)

    # Веса страт
    strata_weights = <...>

    # Размер выборок
    strata_sample_size = <...>

    a_group, b_group = [], []
    # Генерируем для каждой страты отдельно
    for i, strata in enumerate(groupped):
        a, b = random_sampler(strata[1], strata_sample_size[i])
        a_group.append(a)
        b_group.append(b)

    return pd.concat(a_group), pd.concat(b_group)

def weighted_var(data, strata_weights):
    '''
    Вычисляет взвешенную дисперсию при стратификации.

    :param data: данные
    :param strata_weights: веса страт
    :return: дисперсия
    '''

    strata_vars = data.groupby(by='strata')['sample'].var()
    return <...>

def weighted_post_var(data, strata_weights):
    '''
    Вычисляет взвешенную дисперсию при постстратификации.

    :param data: данные
    :param strata_weights: веса страт
    :return: дисперсия
    '''

    strata_vars = data.groupby(by='strata')['sample'].var()
    weighted_var = (strata_vars * strata_weights).sum() / len(data)
    post_addition = (strata_vars * (1-strata_weights)).sum() / (len(data)**2)
    return <...>


def simple_var(data, strata_weights):
    '''
    Вычисляет обычную дисперсию при случайном семплировании.

    :param data: данные
    :param strata_weights: веса страт, не используется, нужен для совместимости
    :return: дисперсия
    '''

    return <...>

In [ ]:
def stratified_ttest(x_data, y_data, strata_weights, method='random', alpha=0.05):
    '''
    Стратифицированный абсолютный t-test.

    :param x_data: выборка вместе с информацией о стратах
    :param y_data: выборка вместе с информацией о стратах
    :param strata_weights: веса страт
    :param method: метод 'random', 'stratified', 'post_stratified'
    :param alpha: уровень значимости
    :return: статистика критерия, границы доверительного интервала, p-value
    '''

    if method == 'random':
        mean_function = simple_mean
        var_function = simple_var
    elif method == 'stratified':
        mean_function = weighted_mean
        var_function = weighted_var

    x_mean = mean_function(x_data, strata_weights)
    y_mean = mean_function(y_data, strata_weights)
    x_var  =  var_function(x_data, strata_weights)
    y_var  =  var_function(y_data, strata_weights)

    stat = <...>
    std = <...>

    t_stat = <...>
    pvalue = <...>

    q = sps.norm.ppf(1 - alpha/2)
    left_bound = <...>
    right_bound = <...>

    return stat, left_bound, right_bound, pvalue

Теперь проверим корректность применения стратификации в нашем случае.

Сначала создадим отдельный датасет для более удобной проверки. У него будет две колонки: `sample`, `strata`, которые соответствуют данным и разделением на страты у исходного датасета.

In [ ]:
data = pd.DataFrame()
data['strata'] = <...>
data['sample'] = <...>
data = data[['strata', 'sample']]
data

Посмотрите на `boxplot` для двух подгрупп.

In [ ]:
sns.boxplot(data, x=<...>, y=<...>)

In [ ]:
data['strata'].sum()

Что вы можете сказать уже сейчас о том, повлияет ли стратификация на результаты теста?

**Ответ:**



In [ ]:
p_vals = []

for i in tqdm(range(2000)):

    control_sample, test_sample = <...>

    strata_weights = <...>
    p_vals.append(<...>)

p_vals = np.array(p_vals)

Посмотрите также на реальный уровень значимости и гистограмму p-value

Корректны ли выбранные методы?

**Ответ:**

Посмотрите на мощность критерия. Улучшилась ли мощность по сравнению с обработкой без стратификации?

In [ ]:
print(f'Реальная мощность критерия: {(pvals < alpha).mean().round(3)}')

**Ответ на вопрос о мощности:**

**Вывод:**

---
### Задача 4
#### **Профиль биология**

Повторите исследование из предыдущей задачи, используя стратификацию по одному из признаков, например, по числу беременностей. Можно попробовать стратификацию по непрерывным признакам, разбив их на категории, если вы считаете такой признак более важным

In [ ]:
DATA_PATH =

In [ ]:
data = pd.read_csv(DATA_PATH)
data = data[data['Outcome']==1]
data

In [ ]:
data.groupby(by=<...>).count().iloc[:, 0]

Можно объединить редкие страты в одну страту.

In [ ]:
<...>

Визуализируйте полученные страты

In [ ]:
pregnancies = data.groupby(by=<...>).count().iloc[:, 0].sort_values(ascending=False)

plt.figure(figsize=(10, 7))
sns.countplot(x=<...>, order=pregnancies.index)
plt.xlabel(<...>)
plt.ylabel('Количество пациентов')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# преобразуем данные для совместимости с кодом функций
data['strata'] = <...>
data['sample'] = <...>
data = data[['strata', 'sample']]
data

Реализуйте функции для стратифицированного семплирования и стратифицированного относительного t-test. Заранее вспомните с семинара, какое семплирование какому тесту подходит. Не забудьте о возможности пост-стратификации. Какие методы подойдут вам, учитывая количество страт и размер выборки?

**Ответ:**

In [ ]:
def calculate_strata_weights(data):
    '''
    Вычисляет веса страт.

    :param data: исходные данные
    :return: веса страт
    '''

    strata_weights = data.groupby(by='strata')['sample'].count()
    strata_weights = strata_weights / strata_weights.sum()
    return <...>


def weighted_mean(data, weights):
    '''
    Вычисляет взвешенное среднее.

    :param data: данные
    :param weights: веса страт
    :return: взвешенное среднее
    '''

    strata_means = data.groupby(by='strata')['sample'].mean()
    return <...>


def simple_mean(data, weights):
    '''
    Вычисляет обычное выборочное среднее

    :param data: данные
    :param weights: не используется, нужен для совместимости
    :return: выборочное среднее
    '''

    return <...>


def random_sampler(data, sample_size):
    '''
    Случайное семплирование двух групп одного размера из данных.

    :param data: исходные данные
    :param sample_size: размер группы
    :return: две группы данных
    '''

    choiced = data.sample(n=2 * sample_size)
    return choiced.iloc[:sample_size], choiced.iloc[sample_size:]

def stratified_sampler(data, sample_size):
    '''
    Стратифицированное семплирование выборок.

    :param data: данные
    :param sample_size: размер выборки
    :return: две выборки
    '''

    groupped = data.groupby(<...>)

    # Веса страт
    strata_weights = <...>

    # Размер выборок
    strata_sample_size = <...>

    a_group, b_group = [], []
    # Генерируем для каждой страты отдельно
    for i, strata in enumerate(groupped):
        a, b = random_sampler(strata[1], strata_sample_size[i])
        a_group.append(a)
        b_group.append(b)

    return pd.concat(a_group), pd.concat(b_group)

def weighted_var(data, strata_weights):
    '''
    Вычисляет взвешенную дисперсию при стратификации.

    :param data: данные
    :param strata_weights: веса страт
    :return: дисперсия
    '''

    strata_vars = data.groupby(by='strata')['sample'].var()
    return <...>

def weighted_post_var(data, strata_weights):
    '''
    Вычисляет взвешенную дисперсию при постстратификации.

    :param data: данные
    :param strata_weights: веса страт
    :return: дисперсия
    '''

    strata_vars = data.groupby(by='strata')['sample'].var()
    weighted_var = (strata_vars * strata_weights).sum() / len(data)
    post_addition = (strata_vars * (1-strata_weights)).sum() / (len(data)**2)
    return <...>


def simple_var(data, strata_weights):
    '''
    Вычисляет обычную дисперсию при случайном семплировании.

    :param data: данные
    :param strata_weights: веса страт, не используется, нужен для совместимости
    :return: дисперсия
    '''

    return <...>

In [ ]:
def stratified_ttest(x_data, y_data, strata_weights, method='random', alpha=0.05):
    '''
    Стратифицированный абсолютный t-test.

    :param x_data: выборка вместе с информацией о стратах
    :param y_data: выборка вместе с информацией о стратах
    :param strata_weights: веса страт
    :param method: метод 'random', 'stratified', 'post_stratified'
    :param alpha: уровень значимости
    :return: статистика критерия, границы доверительного интервала, p-value
    '''

    if method == 'random':
        mean_function = simple_mean
        var_function = simple_var
    elif method == 'stratified':
        mean_function = weighted_mean
        var_function = weighted_var
    elif method == 'post_stratified':
        mean_function = weighted_mean
        var_function = weighted_post_var

    x_mean = mean_function(x_data, strata_weights)
    y_mean = mean_function(y_data, strata_weights)
    x_var  =  var_function(x_data, strata_weights)
    y_var  =  var_function(y_data, strata_weights)

    stat = <...>
    std = <...>

    t_stat = <...>
    pvalue = <...>

    q = sps.norm.ppf(1 - alpha/2)
    left_bound = <...>
    right_bound = <...>

    return stat, left_bound, right_bound, pvalue

In [ ]:
p_vals = []

for i in tqdm(range(2000)):

    control_sample, test_sample = <...>

    strata_weights = <...>
    p_vals.append(<...>)

p_vals = np.array(p_vals)

100%|██████████| 2000/2000 [00:26<00:00, 74.37it/s]


Посмотрите также на реальный уровень значимости и гистограмму p-value

Корректны ли выбранные методы?

**Ответ:**


Посмотрите на мощность критерия. Улучшилась ли мощность по сравнению с прошлой задачей? Не забудьте, что при искусственных АВ-тестах при расчете мощности полезно добавлять шум к данным.

In [ ]:
print(f'Реальная мощность критерия: {(pvals < alpha).mean().round(3)}')

**Ответ:**

**Вывод:**

---
### Задача 5

Рассмотренные нами методы стратификации работают в случае стратифицированного семплирования, либо же случайного семплирования. Во втором случае метод называется постстратификацией. Однако, как мы видели на занятии, при наличии сильных перекосов в соотношении между стратами вероятность ошибки 1 рода может сильно увеличиваться.

Промоделируйте распределение разности
* обычного среднего
* стратифицированного среднего

при наличии сильных перекосов в соотношении между стратами и проинтерпретируйте результаты.

Код для семплирования перекошенных данных и для семплирования разности средних есть в ноутбуке с занятия.

В чем же тогда проблема того, что стратифицированный t-test увеличивает вероятность ошибки 1 рода?

...

Можно ли как-то исправить его с помощью некоторой универсальной статистической процедуры?

...

Реализуйте эту процедруру.

Провалидируйте полученный тест, а также сравните его по мощности со стандартным t-test. Используйте код с занятий.